# News적재

In [3]:
from pathlib import Path
import os

os.chdir(Path(os.getcwd()).parent.parent)

In [4]:
from os.path import join

import json
import ast

import pandas as pd
import numpy as np

In [5]:
root_path = os.getcwd()
data_path = join(os.getcwd(), "data")
raw_path = join(data_path, "raw")
pre_path = join(data_path, "preprocessing")
bulk_path = join(data_path, "bulk")

## Load news_data_20000.csv

In [6]:
target_name = "news"
target_file_name = "news_data_20000.csv"

In [7]:
target_df = pd.read_csv(join(raw_path, target_file_name))

In [8]:
target_df.head()

,id,title,content,provider,category,byline,dateline
0,63792442,"안산시, 생활임금 시급 1만500원 의결",[헤럴드경제(안산)=박정규 기자]안산시(시장 윤화섭) 생활임금위원회는 2022년 생...,헤럴드경제,"['사회>노동_복지', '경제>자동차', '경제>취업_창업']",박정규 기자,2021-09-09T07:44:33.000Z
1,62993103,"울산 하이본 병원, 6월부터 본격 진료 ""척추와 관절 분야 전문""",척추와 관절 분야 전문인 울산 하이본 병원이 6월부터 본격 진료를 시작했다. \n\...,울산매일,"['사회>의료_건강', '지역>울산', '지역>강원']",.,2021-08-05T07:49:07.000Z
2,63800371,금융권 최초 공동 정보지갑서비스 출시…1호 증명서는 '뱅크아이디',[아시아경제 송승섭 기자]금융분산ID 추진 협의회는 국내 최초 정보지갑인 마이인포에...,아시아경제,"['IT_과학>보안', 'IT_과학>모바일', '경제>금융_재테크']",송승섭,2021-09-01T01:34:48.000Z
3,62398473,“총장 후보자 수사심의위 신청은 모순” “정치적 수단 전락”,서울 서초구 대검찰청 검찰기. 연합뉴스 “정치적 중립성을 지키고 수사 결정 전 과정...,세계일보,"['사회>사건_사고', '사회>미디어']",이창훈,2021-04-25T10:18:14.000Z
4,63350588,[인사]현대해상,◆현대해상 ◇임원 전보 △리스크관리본부장 홍사경 △고객지원본부장 황미은 △CCO 윤...,전자신문,"['경제>금융_재테크', '경제>자동차']",김현민,2021-07-01T00:40:53.000Z


In [9]:
target_df.dtypes

id           int64
title       object
content     object
provider    object
category    object
byline      object
dateline    object
dtype: object

## 데이터 변환
- provider
- category
- byline
- dateline

### provider

In [57]:
provider_df = pd.read_csv(join(pre_path, "providers.csv"))
provider_df = provider_df.rename(columns={"이름": "name", "분류": "type", "지역": "location"})

In [58]:
provider_df.head()

,name,type,location
0,헤럴드경제,경제/IT,NaN
1,아시아경제,경제/IT,NaN
2,세계일보,일간지,NaN
3,전자신문,경제/IT,NaN
4,부산일보,지역지,부산


In [75]:
def get_provider_dict(provider: str) -> dict:
    provider_row = provider_df.loc[provider_df["name"] == provider].dropna(axis=1)
    provider_row = provider_row.loc[provider_row.index[0], :]
    return provider_row.to_dict()

In [60]:
test_provider = "헤럴드경제"
get_provider_dict(test_provider)

{'name': '헤럴드경제', 'type': '경제/IT'}

### category

In [82]:
def get_category_dict(category: str) -> list:
    category_list = []
    for elem in ast.literal_eval(category):
        splits = elem.split(">")
        category_list.append({"major": splits[0], "minor": splits[1]})

    return category_list

In [83]:
category = "['사회>노동_복지', '경제>자동차', '경제>취업_창업']"
get_category_dict(category)

[{'major': '사회', 'minor': '노동_복지'},
 {'major': '경제', 'minor': '자동차'},
 {'major': '경제', 'minor': '취업_창업'}]

### byline

"이름" 탭에서 읽어와서 "정규화" 탭에 있는 이름으로 매핑

1. 이름 없는 경우 -> isinstance(name, float)
2. 이름 하나인 경우 -> len(split_names) == 1, split_names[0]
3. 이름 여러 개인 경우 -> split_names

In [10]:
byline_df = pd.read_csv(join(pre_path, "bylines.csv"))

In [11]:
byline_df.head()

,이름,정규화
0,(中）/[번역]강지혜\n[번역]강지혜,강지혜
1,(강원)강대웅·위준휘\n위준휘,강대웅||위준휘
2,(경주) 최주호\n최주호,최주호
3,(과천) 박재천\n박재천,박재천
4,(광주)박승호\n박승호,박승호


In [95]:
def get_reporter_dict(byline):
    if isinstance(byline, float):
        return None
    
    name = byline_df.loc[byline_df["이름"] == byline]["정규화"].values[0]
    
    if isinstance(name, float):
        return None

    split_names = name.split("||")
    
    if len(split_names) == 1:
        return split_names[0]
    
    return split_names

In [38]:
byline = "(강원)강대웅·위준휘\n위준휘"
get_reporter_dict(byline)

['강대웅', '위준휘']

In [40]:
byline = "NEWSFLOW;"
reporter = get_reporter_dict(byline)
type(reporter)

NoneType

### dateline

In [42]:
datelines = target_df["dateline"]

In [43]:
nan_cnt = 0

for index, value in datelines.items():
    if value == np.nan:
        nan_cnt += 1

nan_cnt

0

In [47]:
sample = datelines.values[0]
sample

'2021-09-09T07:44:33.000Z'

## bulk_df

In [52]:
columns = ["id", "title", "content", "provider", "category", "reporter", "dateline"]
bulk_df = pd.DataFrame(columns=columns)

In [53]:
copy_columns = ["id", "title", "content", "dateline"]

for column in copy_columns:
    bulk_df[column] = target_df[column]

bulk_df.shape

(20000, 7)

In [54]:
bulk_df.head()

,id,title,content,provider,category,reporter,dateline
0,63792442,"안산시, 생활임금 시급 1만500원 의결",[헤럴드경제(안산)=박정규 기자]안산시(시장 윤화섭) 생활임금위원회는 2022년 생...,NaN,NaN,NaN,2021-09-09T07:44:33.000Z
1,62993103,"울산 하이본 병원, 6월부터 본격 진료 ""척추와 관절 분야 전문""",척추와 관절 분야 전문인 울산 하이본 병원이 6월부터 본격 진료를 시작했다. \n\...,NaN,NaN,NaN,2021-08-05T07:49:07.000Z
2,63800371,금융권 최초 공동 정보지갑서비스 출시…1호 증명서는 '뱅크아이디',[아시아경제 송승섭 기자]금융분산ID 추진 협의회는 국내 최초 정보지갑인 마이인포에...,NaN,NaN,NaN,2021-09-01T01:34:48.000Z
3,62398473,“총장 후보자 수사심의위 신청은 모순” “정치적 수단 전락”,서울 서초구 대검찰청 검찰기. 연합뉴스 “정치적 중립성을 지키고 수사 결정 전 과정...,NaN,NaN,NaN,2021-04-25T10:18:14.000Z
4,63350588,[인사]현대해상,◆현대해상 ◇임원 전보 △리스크관리본부장 홍사경 △고객지원본부장 황미은 △CCO 윤...,NaN,NaN,NaN,2021-07-01T00:40:53.000Z


### provider

In [76]:
provider_dicts = []
for index, value in target_df["provider"].items():
    provider_dicts.append(get_provider_dict(value))

In [77]:
bulk_df["provider"] = provider_dicts

In [78]:
bulk_df.head()

,id,title,content,provider,category,reporter,dateline
0,63792442,"안산시, 생활임금 시급 1만500원 의결",[헤럴드경제(안산)=박정규 기자]안산시(시장 윤화섭) 생활임금위원회는 2022년 생...,"{'name': '헤럴드경제', 'type': '경제/IT'}",NaN,NaN,2021-09-09T07:44:33.000Z
1,62993103,"울산 하이본 병원, 6월부터 본격 진료 ""척추와 관절 분야 전문""",척추와 관절 분야 전문인 울산 하이본 병원이 6월부터 본격 진료를 시작했다. \n\...,"{'name': '울산매일', 'type': '지역지', 'location': '울산'}",NaN,NaN,2021-08-05T07:49:07.000Z
2,63800371,금융권 최초 공동 정보지갑서비스 출시…1호 증명서는 '뱅크아이디',[아시아경제 송승섭 기자]금융분산ID 추진 협의회는 국내 최초 정보지갑인 마이인포에...,"{'name': '아시아경제', 'type': '경제/IT'}",NaN,NaN,2021-09-01T01:34:48.000Z
3,62398473,“총장 후보자 수사심의위 신청은 모순” “정치적 수단 전락”,서울 서초구 대검찰청 검찰기. 연합뉴스 “정치적 중립성을 지키고 수사 결정 전 과정...,"{'name': '세계일보', 'type': '일간지'}",NaN,NaN,2021-04-25T10:18:14.000Z
4,63350588,[인사]현대해상,◆현대해상 ◇임원 전보 △리스크관리본부장 홍사경 △고객지원본부장 황미은 △CCO 윤...,"{'name': '전자신문', 'type': '경제/IT'}",NaN,NaN,2021-07-01T00:40:53.000Z


### category

In [84]:
category_dicts = []
for index, value in target_df["category"].items():
    category_dicts.append(get_category_dict(value))

In [85]:
bulk_df["category"] = category_dicts

In [86]:
bulk_df.head()

,id,title,content,provider,category,reporter,dateline
0,63792442,"안산시, 생활임금 시급 1만500원 의결",[헤럴드경제(안산)=박정규 기자]안산시(시장 윤화섭) 생활임금위원회는 2022년 생...,"{'name': '헤럴드경제', 'type': '경제/IT'}","[{'major': '사회', 'minor': '노동_복지'}, {'major': ...",NaN,2021-09-09T07:44:33.000Z
1,62993103,"울산 하이본 병원, 6월부터 본격 진료 ""척추와 관절 분야 전문""",척추와 관절 분야 전문인 울산 하이본 병원이 6월부터 본격 진료를 시작했다. \n\...,"{'name': '울산매일', 'type': '지역지', 'location': '울산'}","[{'major': '사회', 'minor': '의료_건강'}, {'major': ...",NaN,2021-08-05T07:49:07.000Z
2,63800371,금융권 최초 공동 정보지갑서비스 출시…1호 증명서는 '뱅크아이디',[아시아경제 송승섭 기자]금융분산ID 추진 협의회는 국내 최초 정보지갑인 마이인포에...,"{'name': '아시아경제', 'type': '경제/IT'}","[{'major': 'IT_과학', 'minor': '보안'}, {'major': ...",NaN,2021-09-01T01:34:48.000Z
3,62398473,“총장 후보자 수사심의위 신청은 모순” “정치적 수단 전락”,서울 서초구 대검찰청 검찰기. 연합뉴스 “정치적 중립성을 지키고 수사 결정 전 과정...,"{'name': '세계일보', 'type': '일간지'}","[{'major': '사회', 'minor': '사건_사고'}, {'major': ...",NaN,2021-04-25T10:18:14.000Z
4,63350588,[인사]현대해상,◆현대해상 ◇임원 전보 △리스크관리본부장 홍사경 △고객지원본부장 황미은 △CCO 윤...,"{'name': '전자신문', 'type': '경제/IT'}","[{'major': '경제', 'minor': '금융_재테크'}, {'major':...",NaN,2021-07-01T00:40:53.000Z


### reporter

In [96]:
reporter_dicts = []
for index, value in target_df["byline"].items():
    reporter_dicts.append(get_reporter_dict(value))

In [97]:
bulk_df["reporter"] = reporter_dicts

In [98]:
bulk_df.head()

,id,title,content,provider,category,reporter,dateline
0,63792442,"안산시, 생활임금 시급 1만500원 의결",[헤럴드경제(안산)=박정규 기자]안산시(시장 윤화섭) 생활임금위원회는 2022년 생...,"{'name': '헤럴드경제', 'type': '경제/IT'}","[{'major': '사회', 'minor': '노동_복지'}, {'major': ...",박정규,2021-09-09T07:44:33.000Z
1,62993103,"울산 하이본 병원, 6월부터 본격 진료 ""척추와 관절 분야 전문""",척추와 관절 분야 전문인 울산 하이본 병원이 6월부터 본격 진료를 시작했다. \n\...,"{'name': '울산매일', 'type': '지역지', 'location': '울산'}","[{'major': '사회', 'minor': '의료_건강'}, {'major': ...",None,2021-08-05T07:49:07.000Z
2,63800371,금융권 최초 공동 정보지갑서비스 출시…1호 증명서는 '뱅크아이디',[아시아경제 송승섭 기자]금융분산ID 추진 협의회는 국내 최초 정보지갑인 마이인포에...,"{'name': '아시아경제', 'type': '경제/IT'}","[{'major': 'IT_과학', 'minor': '보안'}, {'major': ...",송승섭,2021-09-01T01:34:48.000Z
3,62398473,“총장 후보자 수사심의위 신청은 모순” “정치적 수단 전락”,서울 서초구 대검찰청 검찰기. 연합뉴스 “정치적 중립성을 지키고 수사 결정 전 과정...,"{'name': '세계일보', 'type': '일간지'}","[{'major': '사회', 'minor': '사건_사고'}, {'major': ...",이창훈,2021-04-25T10:18:14.000Z
4,63350588,[인사]현대해상,◆현대해상 ◇임원 전보 △리스크관리본부장 홍사경 △고객지원본부장 황미은 △CCO 윤...,"{'name': '전자신문', 'type': '경제/IT'}","[{'major': '경제', 'minor': '금융_재테크'}, {'major':...",김현민,2021-07-01T00:40:53.000Z


### change column name
- dateline → date

In [99]:
bulk_df = bulk_df.rename(columns={"dateline": "date"})

In [100]:
bulk_df.head()

,id,title,content,provider,category,reporter,date
0,63792442,"안산시, 생활임금 시급 1만500원 의결",[헤럴드경제(안산)=박정규 기자]안산시(시장 윤화섭) 생활임금위원회는 2022년 생...,"{'name': '헤럴드경제', 'type': '경제/IT'}","[{'major': '사회', 'minor': '노동_복지'}, {'major': ...",박정규,2021-09-09T07:44:33.000Z
1,62993103,"울산 하이본 병원, 6월부터 본격 진료 ""척추와 관절 분야 전문""",척추와 관절 분야 전문인 울산 하이본 병원이 6월부터 본격 진료를 시작했다. \n\...,"{'name': '울산매일', 'type': '지역지', 'location': '울산'}","[{'major': '사회', 'minor': '의료_건강'}, {'major': ...",None,2021-08-05T07:49:07.000Z
2,63800371,금융권 최초 공동 정보지갑서비스 출시…1호 증명서는 '뱅크아이디',[아시아경제 송승섭 기자]금융분산ID 추진 협의회는 국내 최초 정보지갑인 마이인포에...,"{'name': '아시아경제', 'type': '경제/IT'}","[{'major': 'IT_과학', 'minor': '보안'}, {'major': ...",송승섭,2021-09-01T01:34:48.000Z
3,62398473,“총장 후보자 수사심의위 신청은 모순” “정치적 수단 전락”,서울 서초구 대검찰청 검찰기. 연합뉴스 “정치적 중립성을 지키고 수사 결정 전 과정...,"{'name': '세계일보', 'type': '일간지'}","[{'major': '사회', 'minor': '사건_사고'}, {'major': ...",이창훈,2021-04-25T10:18:14.000Z
4,63350588,[인사]현대해상,◆현대해상 ◇임원 전보 △리스크관리본부장 홍사경 △고객지원본부장 황미은 △CCO 윤...,"{'name': '전자신문', 'type': '경제/IT'}","[{'major': '경제', 'minor': '금융_재테크'}, {'major':...",김현민,2021-07-01T00:40:53.000Z


## Bulk

In [101]:
action_str = {
    "index": {
        "_index": target_name
    }
}

action = json.dumps(action_str)

In [110]:
def get_row_dict(row):
    return {"id": row["id"], "title": row["title"], "content": row["content"],
            "provider": row["provider"], "category": row["category"], "reporter": row["reporter"],
            "date": row["date"]}

In [111]:
bulk_data = ""

count = 0
for index, row in bulk_df.iterrows():
    row_dict = get_row_dict(row)
    row_bulk = json.dumps(row_dict, ensure_ascii=False)
    bulk_data += f"{action}\n{row_bulk}\n"

In [112]:
with open(join(bulk_path, f"{target_name}.json"), "w", encoding="utf-8") as json_file:
    json_file.write(bulk_data)